In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

In [13]:
tempIemisions = pd.read_csv("DataSetFinal.csv", index_col=0,na_values = ['no info', '.'])


         dt          Value         Country     Type  AverageTemperature  Mes
0      1990   62297.166000         Austria      Co2              -2.443    1
1      1990    4293.063000         Austria      N2O              -2.443    1
2      1990   10598.566000         Austria  Methane              -2.443    1
3      1990   63417.203939         Austria      Co2               2.713    2
4      1990    4342.948570         Austria      N2O               2.713    2
5      1990   10608.935464         Austria  Methane               2.713    2
6      1990   64354.776643         Austria      Co2               5.391    3
7      1990    4385.020950         Austria      N2O               5.391    3
8      1990   10614.433618         Austria  Methane               5.391    3
9      1990   65120.698769         Austria      Co2               5.462    4
10     1990    4419.699865         Austria      N2O               5.462    4
11     1990   10615.394522         Austria  Methane               5.462    4

In [14]:
tempIemisions.describe()

,dt,Value,AverageTemperature,Mes
count,22356.000000,2.235600e+04,22356.000000,22356.000000
mean,2001.000000,6.270248e+04,9.230230,6.832797
std,6.633398,1.379788e+05,9.537685,3.484143
min,1990.000000,6.148900e+01,-34.665000,1.000000
25%,1995.000000,5.287999e+03,3.819500,4.000000
50%,2001.000000,1.352975e+04,10.129500,7.000000
75%,2007.000000,5.538075e+04,16.188250,10.000000
max,2012.000000,1.052238e+06,28.916000,13.000000


In [19]:
#fem X totes les columnes menys temp i y nomes temp el axis=1 es per indicar
#que es una label de les columnes
X = tempIemisions.drop('AverageTemperature', axis=1)
y = tempIemisions['AverageTemperature']

#dividim el dataSet en test 20% i entrenament 80%
#el random_state es la llavor que s'utilitzarà per randomitzar 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#creem l'algorisme del arbre de decisions i l'entrenem
regressor = DecisionTreeRegressor()
regressor.fit(X_train,y_train)

ValueError: could not convert string to float: 'Methane'